In [ ]:
"""_summary_
df_vehicle  : vehicle spec csv file
                assumptions:
                + year affects it ? no becuase not a primary key
df_0        : parquet file 1 for driving data - raw
df_0_1      : df_0 with date and time seperated into seperate columns 
                assumptions: 
                + duplicate times / vehicle spec ids - do a distinct at the start (if necessary after confirming)
                + trip duration : max_time - min_time per vehicle, on that 
                    + vehicle was running continuously without stopping 
                    + can we figure out if it did stop? exclude velocity = 0/null data
                + distance travelled: distance between the lat/long of max_time and min_time
                    + this is the displacement not the distance
                    + to improve : use a lag/lead to calculate the displacement between each timestamp update per vehicle and sum up total distance travelled 




"""

In [ ]:
import pandas as pd
import pyarrow.parquet as pa
from pathlib import Path
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window





In [ ]:
data_path = '/Users/athulparvelikudy/Personal/ACRTA/tech-test-data/supporting-data'
file_name_0 = 'drive/part-00000-tid-4109877695252048813-a3139a95-1807-419c-af03-4877385b4c8c-11-1-c000.snappy.parquet'
file_name_1 = 'drive/part-00001-tid-4109877695252048813-a3139a95-1807-419c-af03-4877385b4c8c-12-1-c000.snappy.parquet'
file_name_2 = 'drive/part-00002-tid-4109877695252048813-a3139a95-1807-419c-af03-4877385b4c8c-13-1-c000.snappy.parquet'
file_name_3 = 'drive/part-00003-tid-4109877695252048813-a3139a95-1807-419c-af03-4877385b4c8c-14-1-c000.snappy.parquet'
file_name_4 = 'drive/part-00004-tid-4109877695252048813-a3139a95-1807-419c-af03-4877385b4c8c-15-1-c000.snappy.parquet'
vehicle_file = 'vehicle.csv'

vehicle_csv = str(Path(data_path) / vehicle_file)

full_file_path0 = str(Path(data_path) / file_name_0)
full_file_path1 = str(Path(data_path) / file_name_1)
full_file_path2 = str(Path(data_path) / file_name_2)
full_file_path3 = str(Path(data_path) / file_name_3)
full_file_path4 = str(Path(data_path) / file_name_4)



In [ ]:


spark = (
    SparkSession.builder
    .appName("ReadLocalParquet")
    .getOrCreate()
)


In [ ]:
spark.version

In [ ]:
df_0 = spark.read.parquet(full_file_path0)


In [ ]:
df_0.printSchema()
df_0.count()


In [ ]:
df_vehicle = (
    spark.read
    .option("header", "true") 
    .option("inferSchema", "true") 
    .csv(vehicle_csv)
)

df_vehicle.printSchema()
df_vehicle.count()


In [ ]:
print("Parquet file (10 rows):")
df_0.show(10, truncate=False)

print("CSV file (10 rows):")
df_vehicle.show(10, truncate=False)


In [ ]:
df_0_1 = (
    df_0
    .withColumn("date", F.to_date("datetime"))
    .withColumn("time", F.date_format("datetime", "HH:mm:ss"))
)

In [ ]:
input_dt = "2017-01-07"
# TODO: make a check to see if the input is in the correct format before processing

df_2017_01_06 = df_0_1.filter(F.col("date") == F.lit(input_dt))

df_2017_01_06.count() # around 300k

# df_2017_01_06.show(20, truncate=False)

# df_2017_01_06.orderBy(F.col("time").desc()).show(20, truncate=False)

df_2017_01_06.orderBy("time", ascending=False).limit(5).show(truncate=False)

df_joined = (
    df_0_1
    .filter(F.col("date") == F.to_date(F.lit(input_dt)))
    .join(df_vehicle, on="vehicle_spec_id", how="left")
).limit(5).show()

In [ ]:


df_trip_duration_1 = (
    df_2017_01_06
    .groupBy("vehicle_spec_id", "trip_id")
    .agg(
        F.min("datetime").alias("min_datetime"),
        F.max("datetime").alias("max_datetime"),
    )
    .withColumn("trip_duration_seconds", F.unix_timestamp("max_datetime") - F.unix_timestamp("min_datetime"))
    .select( "trip_id", "vehicle_spec_id", "trip_duration_seconds")
)

df_trip_duration_2 = (
    df_2017_01_06
    .groupBy("vehicle_spec_id")
    .agg(
        F.min("datetime").alias("min_datetime"),
        F.max("datetime").alias("max_datetime"),
    )
    .withColumn("trip_duration_seconds", F.unix_timestamp("max_datetime") - F.unix_timestamp("min_datetime"))
    .select( "vehicle_spec_id", "trip_duration_seconds")
)


# df_trip_duration.show(20, truncate=False)
vehicle_cnt = df_2017_01_06.select("vehicle_spec_id").distinct().count()
print(f"df_2017_01_06 vehicles: {vehicle_cnt}")

trip_cnt = df_2017_01_06.select("vehicle_spec_id", "trip_id").distinct().count()
print(f"df_2017_01_06 trips: {trip_cnt}")

print(f'df_trip_duration_1: {df_trip_duration_1.count()}')
print(f'df_trip_duration_2: {df_trip_duration_2.count()}')

df_trip_duration_1.select("vehicle_spec_id", "trip_id").distinct().count()
df_trip_duration_2.select("vehicle_spec_id").distinct().count()


In [ ]:
df_duration = (
    df_2017_01_06
    .groupBy("vehicle_spec_id")
    .agg(
        F.min("datetime").alias("min_datetime"),
        F.max("datetime").alias("max_datetime"),
    )
    .withColumn("trip_duration_seconds", F.unix_timestamp("max_datetime") - F.unix_timestamp("min_datetime"))
    .select( "vehicle_spec_id", "trip_duration_seconds")
)

w = Window.partitionBy("vehicle_spec_id").orderBy(F.col("datetime").asc())

df_min_trip = (
    df_2017_01_06
    .withColumn("rn", F.row_number().over(w))
    .filter(F.col("rn") == 1)
    .select(
        "vehicle_spec_id",
        F.col("trip_id").alias("min_time_trip_id")
    )
)

df_trip_duration = (
    df_duration
    .join(df_min_trip, on="vehicle_spec_id", how="left")
    .select("vehicle_spec_id", "min_time_trip_id", "trip_duration_seconds")
)

df_trip_duration.count()


In [ ]:
w_asc  = Window.partitionBy("trip_id").orderBy(F.col("datetime").asc())
w_desc = Window.partitionBy("trip_id").orderBy(F.col("datetime").desc())

df_start = (
    df_2017_01_06
    .withColumn("rn", F.row_number().over(w_asc))
    .filter(F.col("rn") == 1)
    .select(
        "trip_id",
        "vehicle_spec_id",
        F.col("datetime").alias("start_datetime"),
        F.col("lat").alias("start_lat"),
        F.col("long").alias("start_long"),
    )
)

df_end = (
    df_2017_01_06
    .withColumn("rn", F.row_number().over(w_desc))
    .filter(F.col("rn") == 1)
    .select(
        "trip_id",
        F.col("datetime").alias("end_datetime"),
        F.col("lat").alias("end_lat"),
        F.col("long").alias("end_long"),
    )
)

df_trip_summary = (
    df_start.alias("s")
    .join(df_end.alias("e"), on="trip_id", how="inner")
    .withColumn(
        "trip_duration_seconds",
        F.unix_timestamp("end_datetime") - F.unix_timestamp("start_datetime")
    )
    # Haversine distance (km) between start and end coordinates
    .withColumn("start_lat_r", F.radians("start_lat"))
    .withColumn("end_lat_r", F.radians("end_lat"))
    .withColumn("dlat", F.radians(F.col("end_lat") - F.col("start_lat")))
    .withColumn("dlon", F.radians(F.col("end_long") - F.col("start_long")))
    .withColumn(
        "a",
        F.pow(F.sin(F.col("dlat") / 2), 2)
        + F.cos("start_lat_r") * F.cos("end_lat_r") * F.pow(F.sin(F.col("dlon") / 2), 2)
    )
    .withColumn("c", 2 * F.atan2(F.sqrt("a"), F.sqrt(1 - F.col("a"))))
    .withColumn("distance_km", F.lit(6371.0) * F.col("c"))
    .drop("start_lat_r", "end_lat_r", "dlat", "dlon", "a", "c")
)

df_trip_summary.show(5, truncate=False)


In [ ]:
df_final = (
    df_trip_summary
    .join(
        df_vehicle.select("vehicle_spec_id", "make", "model"),
        on="vehicle_spec_id",
        how="left"
    )
)

df_final.select(
    "trip_id",
    "end_datetime",
    "trip_duration_seconds",
    "distance_km",
    "make",
    "model"
).show(10, truncate=False)

df_final.select(
    "trip_id",
    F.to_date(F.from_utc_timestamp("end_datetime", "America/Los_Angeles")).alias("date_pst"),
    "trip_duration_seconds",
    "distance_km",
    "make",
    "model"
).show(10, truncate=False)




In [ ]:
"""df0 = pa.read_table(full_file_path0)
print(df0.shape)
df0.slice(0, 10).to_pandas() 

df1 = pa.read_table(full_file_path1)
print(df1.shape)
df1.slice(0, 10).to_pandas()

df2 = pa.read_table(full_file_path2)
print(df2.shape)
df2.slice(0, 10).to_pandas()

df3 = pa.read_table(full_file_path3)
print(df3.shape)
df3.slice(0, 10).to_pandas()

df4 = pa.read_table(full_file_path4)
print(df4.shape)
df4.slice(0, 10).to_pandas()
"""